# Preprocess subject data

In [1]:
import sys
import os
from pathlib import Path
src_path = os.path.dirname(os.path.abspath("../../src"))
sys.path.append(src_path)
from src.preprocessing.classes import Subject, Group
from src.utils.config import Config
import src.configs.config as configs
import src.preprocessing.utils as pre_utils 


imported src module
imported preprocessing module
imported configs module
imported utils module


Load in subject IDs and paths from CFGLog

In [2]:
config = Config.from_json(configs.CFGLog)

CLBP_CP_subject_ids = config.data.chronic_low_back_pain.subject_ids.CP
CLBP_HC_subject_ids = config.data.chronic_low_back_pain.subject_ids.HC
Panc_subject_ids = config.data.chronic_pancreatitis.subject_ids.CP
Lupus_CP_subject_ids = config.data.lupus.subject_ids.CP
Lupus_NP_subject_ids = config.data.lupus.subject_ids.NP

In [3]:
times_tup, time_win_path = pre_utils.get_time_window(configs.PERISTIM_TIME_WIN)

[-2.5,0.0,2.5]


Combine subject IDs

In [4]:
# Combine all subject IDs into dict separated by group
sub_ids = {
    "chronic_low_back_pain": CLBP_CP_subject_ids + CLBP_HC_subject_ids,
    "chronic_pancreatitis": Panc_subject_ids,
    "lupus": Lupus_CP_subject_ids + Lupus_NP_subject_ids
}

print(f"Total subjects: {len(sub_ids['chronic_low_back_pain']) + len(sub_ids['chronic_pancreatitis']) + len(sub_ids['lupus'])}")
print(sub_ids)

Total subjects: 63
{'chronic_low_back_pain': ['018', '022', '024', '031', '032', '034', '036', '039', '040', '045', '046', '052', '020', '021', '023', '029', '037', '041', '042', '044', '048', '049', '050', '056', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C2.', 'C24', 'C25', 'C26', 'C27', 'C3.', 'C6.', 'C7.', 'C9.'], 'chronic_pancreatitis': ['002', '003', '006', '007', '008', '009', '010', '011', '012', '013'], 'lupus': ['5186', '6310', '5295', '5873', '6100', '6106', '5648', '5675', '5845', '5713']}


Define preprocessing parameters

In [7]:
TIME_RANGE = (-2.5, 2.5)
PERISTIM_TIME_WIN = abs(TIME_RANGE[0]) + abs(TIME_RANGE[1])
# BASELINE = (-0.2, 0)

Create Subject object for each subject

In [6]:
for group in sub_ids:
    for sub_id in sub_ids[group]:
        subject = Subject(sub_id, group)
        print(subject.subject_folder)
        subject.preprocess()
        subject.get_cleaned_epochs(TIME_RANGE, BASELINE)           
        break

        # subject.get_cleaned_resting()

        # subject.get_stc_resting()
        # subject.get_stc_epochs()


../../../../George Kenefati/Chronic Low Back Pain Study/Data/Raw/018.EEGDATA


AttributeError: 'Subject' object has no attribute 'raw'

In [ ]:
# Create an instance of SubjectProcessor
processor = SubjectProcessor(paths_dict, roi_acronyms)

# Loop through each subject and process the response
for sub_id in sub_ids_focus:
    subject = Subject(sub_id)
    processor.plot_TFR_and_trace(subject,
                                 channel=channel, 
                                 baseline=baseline,
                                 time_range=time_range,
                                 vlim=vlim,
                                 )
    # processor.process_response(subject)
    # break
    

# Display the results
processor.display_results()
